In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale,Normalizer,OneHotEncoder
from sklearn.preprocessing import normalize
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import keras.optimizers 

In [31]:
train = pd.read_csv('task2.training.csv')

In [32]:
test = pd.read_csv('task2.test.withOUT.answers.csv')

In [33]:
train_answer = train['class']

In [34]:
train['class']

0       B
1       F
2       E
3       E
4       C
       ..
9995    A
9996    D
9997    D
9998    C
9999    D
Name: class, Length: 10000, dtype: object

In [35]:
train = train.drop(columns = 'class')

In [36]:
train = np.array(train,dtype=np.float32)
test = np.array(test,dtype=np.float32)

In [37]:
meanlist=[]
stdlist =[]

In [38]:
for i in range(np.shape(train)[1]):
    tmp = np.sum(train[:,i])
    mean = tmp/np.shape(train)[0]
    meanlist.append(mean)
    std = np.std(train[:,i])
    stdlist.append(std)
    for j in range(np.shape(train)[0]):
        train[j][i] = (train[j][i]-mean)/std

In [39]:
meanlist

[13.92298125,
 10.380321875,
 14.45909375,
 10.558634375,
 10.66776875,
 2.5094435546875,
 3.75401640625,
 14.0880421875,
 10.3049015625,
 14.5248953125,
 -0.920540625,
 -2.55181328125,
 5.046812109375,
 1.48286728515625,
 -0.562686181640625,
 4.48297578125]

In [40]:
stdlist

[6.874074,
 2.9697495,
 5.2536054,
 5.018302,
 4.9927273,
 1.0386636,
 1.5589948,
 6.9650598,
 2.9687245,
 5.297438,
 8.567429,
 6.0524693,
 5.4038343,
 0.8848061,
 3.5752804,
 3.618423]

In [41]:
# normalizer = Normalizer().fit(train) 
# train = normalizer.transform(train)

In [42]:
np.shape(train[:,1])

(10000,)

In [43]:
ans = pd.get_dummies(train_answer)

In [44]:
ans = np.array(ans)

In [45]:
train_feature_len = np.shape(train)[1]

In [46]:
train_x,val_x,train_y,val_y = train_test_split(train,ans,test_size=0.3,random_state=44)

In [47]:
np.shape(train_x)

(7000, 16)

In [19]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [65]:
lr= 1e-2
opti =keras.optimizers.Adam(learning_rate=lr)
model3 = Sequential()  
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*3, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*3, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(6, activation = 'softmax',use_bias=True))
para3 = model3.compile(optimizer = opti, loss = 'categorical_crossentropy', metrics = ['acc',f1_m,precision_m, recall_m])


In [66]:
# model3=load_model('ML_task2.h5')

In [362]:
print('Training ------------')
model3.fit(train_x, train_y, epochs=1,batch_size=128) #3500best

Training ------------
55/55 [==============================] - 0s 1ms/step - loss: 0.0455 - acc: 0.9810 - f1_m: 0.9810 - precision_m: 0.9810 - recall_m: 0.9810


In [363]:
# model3.save('ML_task2.h5')

In [364]:
print('\nTesting ------------')
loss, accuracy, f1_score, precision, recall = model3.evaluate(val_x, val_y, verbose=0)
print('\ntest loss: ', loss)
print('\ntest f1_score: ', f1_score)#0.0.9936835169792175


Testing ------------

test loss:  0.0429210290312767

test f1_score:  0.9827127456665039


In [317]:
test.shape

(2000, 16)

In [318]:
for i in range(np.shape(test)[1]):
    mean = meanlist[i]
    std  = stdlist[i]
    for j in range(np.shape(test)[0]):
        test[j][i] = (test[j][i]-mean)/std

In [319]:
np.sum(test[:,1])

20488.963

In [320]:
prediction = model3.predict(test)

In [321]:
np.argmax(prediction[0], axis=0)

3

In [322]:
alph = ['A','B','C','D','E','F']

In [323]:
prediction_final=[]

In [324]:
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [60]:
# prediction_final

In [326]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])

In [327]:
tt = pd.DataFrame(tt)

In [328]:
tt.to_csv('test.csv',index=False)

In [20]:
## sklearn

In [79]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

clf = MLPClassifier(random_state=1,hidden_layer_sizes=(train_feature_len,train_feature_len*2,train_feature_len*3,train_feature_len*2,train_feature_len,75),learning_rate_init=0.0001, max_iter=1000,learning_rate = 'adaptive').fit(train_x, train_y)
clf.predict_proba(val_x)

clf.predict(val_x)

clf.score(val_x,val_y)

0.971

In [52]:
prediction = clf.predict(test)

In [ ]:
for i in range(np.shape(test)[1]):
    mean = meanlist[i]
    std  = stdlist[i]
    for j in range(np.shape(test)[0]):
        test[j][i] = (test[j][i]-mean)/std

In [53]:
np.argmax(prediction[0], axis=0)

0

In [54]:
alph = ['A','B','C','D','E','F']

In [56]:
prediction_final=[]
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [57]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])

In [58]:
tt = pd.DataFrame(tt)

In [59]:
tt.to_csv('test.csv',index=False)